Power Analysis Data Prep Sequence
Rowan Converse, last update 10/17/2023

Requirements: 
Drone image directory
A GIS software or Agisoft Metashape
DJI Drones only: EXIF CSV output from EXIFTOOL

Manual steps will be noted in markdown

In [68]:
#1: Clip circles (detection radius) from all images; preserve original image metadata

import os
from PIL import Image, ImageDraw

def clip_circle(input_image_path, output_dir):
    # Open the input image
    image = Image.open(input_image_path)

    # Get image dimensions
    width, height = image.size

    # Calculate the center of the image
    center_x = width // 2
    center_y = height // 2

    # Calculate the radius as the shortest distance to the image edge
    radius = min(center_x, center_y)

    # Create a mask to represent the circular clip
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((center_x - radius, center_y - radius, center_x + radius, center_y + radius), fill=255)

    # Apply the circular mask to the image
    result = Image.new('RGB', (width, height))
    result.paste(image, mask=mask)

    # Preserve EXIF data
    result.info = image.info

    # Construct the output image name
    output_image_name = os.path.splitext(os.path.basename(input_image_path))[0] + ".jpg"
    output_image_path = os.path.join(output_dir, output_image_name)

    # Save the result with preserved EXIF data
    result.save(output_image_path, exif=result.info['exif'])

if __name__ == "__main__":
    input_dir = "E:\\poweranalysis\\data\\originals\\17b\\"
    output_dir = "E:\\poweranalysis\\data\\circles\\17b\\"

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # List all files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]

    for image_file in image_files:
        input_image_path = os.path.join(input_dir, image_file)
        clip_circle(input_image_path, output_dir)


Manual step: derive transect IDs
In Agisoft, add the images and align them
Alternately, in a GIS, add the EXIF csv and add the image centers as XY data
Add a new column for transect ID to the output of step 2
Manually note the image names of the start and end of each flightline; use this to designate a transect ID for all images

Use an image select script to separate out the circle clips into folders based off transects
With the known side overlap (flightplan, or manually derived from Agisoft footprints_to_shapes.py + GIS overlap of sample image footprints), preserve the appropriate number of independent side-lap transects before moving on to the next step.

In [6]:
# 2a- Derive image metadata (AGL + GSD): DJI drones 

#Using a csv output from ExifTool that contains Relative Altitude (DJI drones only, so far)
import csv
# Define sensor parameters (in millimeters)-- this is Mavic 2 Pro
image_width = 5472  # Example image width in pixels
image_height = 3648  # Example image height in pixels
sensor_width = 0.0128  # Make sure this is converted to meters!
sensor_height = 0.0096  # Make sure this is converted to meters!
focal_length = 0.01035  # Make sure this is converted to meters!

altitude_data = []  # List to store extracted altitude values

#Requires CSV output from ExifTool
with open('E:\\poweranalysis\\data\\circles\\RGNC\\exif.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        # Extract the "RelativeAltitude" and "FileName" values
        relative_altitude = row.get("RelativeAltitude", None)
        image_filename = row.get("FileName", None)

        if relative_altitude is not None and relative_altitude.strip() != '' and image_filename is not None:
            try:
                relative_altitude = float(relative_altitude)
                if relative_altitude > 120:
                    # Flag values exceeding 120 as likely absolute altitudes
                    print(f"Skipped: {image_filename} - RelativeAltitude exceeds legal flight ceiling: {relative_altitude}")
                else:
                    altitude_data.append((image_filename, relative_altitude, None))
            except ValueError:
                print(f"Skipping: {image_filename} - Invalid RelativeAltitude value: {relative_altitude}")

# Calculate GSD and write data to a new CSV file
with open('E:\\poweranalysis\\data\\circles\\RGNC\\RGNC_gsd.csv', mode='w', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    csv_writer.writerow(["filename", "agl", "gsd"])
    
    for image_filename, relative_altitude in altitude_data:
        # Calculate GSD (in cm)
        gsd_m = (sensor_width * relative_altitude) / (focal_length * image_width)
        gsd_cm = gsd_m * 100  # Convert to cm

        # Write data to the new CSV file
        csv_writer.writerow([image_filename, relative_altitude, gsd_cm])

In [80]:
#2b: Derive AGL/GSD, all other drones
#Assuming you have a CSV file with EXIF data that has absolute altitude

import pandas as pd 

ground_elevation = 1371 #Adjust to the appropriate value for your study area; I typically use the recorded flight absolute altitude of the first image minus the idealized AGL programmed for the flight

# Define sensor parameters
#SONY RXR1 II
image_width = 7952  # Example image width in pixels
image_height = 5304  # Example image height in pixels
sensor_width = 0.0359   # Make sure this is converted to meters!
sensor_height = 0.024  # Make sure this is converted to meters!
focal_length = 0.035 # Make sure this is converted to meters!

#Mavic 2 Pro
#image_width = 5472  # Example image width in pixels
#image_height = 3648  # Example image height in pixels
#sensor_width = 0.0128  # Make sure this is converted to meters!
#sensor_height = 0.0096  # Make sure this is converted to meters!
#focal_length = 0.01035  # Make sure this is converted to meters!

# Read the CSV file
df = pd.read_csv("E:\\poweranalysis\\data\\circles\\40_ind_sides.csv")

# Calculate and store GSD values in a new column
df['gsd'] = (sensor_width * (df['agl'] - ground_elevation)) / (focal_length * image_width)

# Save the updated DataFrame to a new CSV file
df.to_csv("E:\\poweranalysis\\data\\circles\\40_gsd.csv", index=False)

In [84]:
#3a: Derive independent forward overlap images (must use a known overlap value)

import pandas as pd

# Load the CSV file into a pandas dataframe
input_file = 'E:\\poweranalysis\\data\\circles\\40_gsd.csv'  # CSV file that includes image names and transect numbers, ideally filtered by desired transects
df = pd.read_csv(input_file)

# Create a new dataframe with every other row from the original dataframe; can replace "2" with any n value of images to preserve
subset_df = df.iloc[::2].copy()

# Optional: Reset the index of the new dataframe
subset_df.reset_index(drop=True, inplace=True)

# Save the subset dataframe to a new CSV file
output_file = 'E:\\poweranalysis\\data\\circles\\40_ind_img.csv'  # Replace with the desired filename
subset_df.to_csv(output_file, index=False)

In [86]:
#3b: concatenate the different sites, and copy the independent images into a common directory for detection step

# List of CSV file paths to merge
csv_files = ['E:\\poweranalysis\\data\\circles\\6_ind_img.csv', 'E:\\poweranalysis\\data\\circles\\17a_ind_img.csv', 'E:\\poweranalysis\\data\\circles\\17b_ind_img.csv', 'E:\\poweranalysis\\data\\circles\\40_ind_img.csv']  # Add your file paths

# Initialize an empty DataFrame
merged_df = pd.DataFrame()

# Loop through each CSV file and append it to the merged DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('E:\\poweranalysis\\data\\circles\\BdA_2023_ind_imgs.csv', index=False)


In [91]:
#3c- Copy the independent images into a new directory for batch detection

import os
import shutil

# Directory where your images are stored
image_directory = 'E:\\poweranalysis\\data\\circles\\all_circles'

# Directory where you want to copy the matching images
output_directory = 'E:\\poweranalysis\\data\\circles\\BdA2023_detection'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Read the merged CSV file
df = pd.read_csv('E:\\poweranalysis\\data\\circles\\BdA_2023_ind_imgs.csv')

# Create a set of all filenames from the CSV in lowercase for case-insensitive matching
csv_filenames = set(merged_df['filename'].str.lower())

# Walk through the image directory and its subdirectories
for root, dirs, files in os.walk(image_directory):
    for file in files:
        if file.lower() in csv_filenames:
            source_path = os.path.join(root, file)
            target_path = os.path.join(output_directory, file)
            shutil.copy(source_path, target_path)


MANUAL STEP: Land/no land
2 ways to do this

1- If there are a lot of images: Build an orthomosaic in Agisoft with all images. Then project the ortho in a GIS, digitize a boundary of the shoreline. Project the image center XY data. Derive an average image height using the average GSD. Using this info, create an interior buffer from the polygon using that measurement. Select all image centers within the buffer. Export this information-- this is your "no land" set

2- If there aren't a lot of images: just examine the circle clip thumbnails in the Windows Explorer and note any with land in the CSV manually.

In [92]:
#4: Batch tiled detections using SAHI tools

import os
from sahi.utils.yolov5 import download_yolov5s6_model
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image
import pandas as pd

def process_images_in_directory(master_directory, save_directory):
    os.makedirs(save_directory, exist_ok=True)
    yolov5_model_path = 'E:\\training\\Base\\YOLOweights\\zooniverse_3class_150epc\\weights\\best.pt' # specify the full model path
    download_yolov5s6_model(destination_path=yolov5_model_path)

    detection_model = AutoDetectionModel.from_pretrained(
        model_type='yolov5', # can accommodate other architectures
        model_path=yolov5_model_path,
        confidence_threshold=0.8, # specify the confidence threshold as desired
        device="cuda:0" # to use GPU
    )

    all_anns = []

    for root, dirs, files in os.walk(master_directory):
        for img_filename in files:
            if img_filename.lower().endswith(('.tif', '.jpg', '.jpeg', '.png', '.bmp')):
                img_path = os.path.join(root, img_filename)

                result = get_sliced_prediction(
                    img_path,
                    detection_model,
                    slice_height=640, # Can adjust tile size
                    slice_width=640,
                    overlap_height_ratio=0.1, # Can adjust tile overlap
                    overlap_width_ratio=0.1
                )

                coco = result.to_coco_annotations()
                for annotation in coco:
                    annotation['image_id'] = img_filename

                all_anns.extend(coco)

    df = pd.DataFrame(all_anns)
    savepath = os.path.join(save_directory, "BdA2023_detections.csv") # change the name if desired
    df.to_csv(savepath, index=False)

image_directory = "E:\\poweranalysis\\data\\circles\\BdA2023_detection"  # Replace with the path to the master directory containing images
save_directory = "E:\\poweranalysis\\data\\circles"  # Replace with the desired output directory for the COCO annotations
process_images_in_directory(image_directory, save_directory)


YOLOv5  2023-7-27 Python-3.10.9 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 214 layers, 7027720 parameters, 0 gradients, 16.0 GFLOPs
Adding AutoShape... 


Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing prediction on 140 number of slices.
Performing pr

In [103]:
#5- Distance from bird detections to the drone

#Fixed to use altitude/GSD from each individual image to calculate ground distance from drone of individual annotations

import pandas as pd
import ast

# Read the two CSV files
df_annotations = pd.read_csv("E:\\poweranalysis\\data\\circles\\BdA2023_detections.csv") #COCO format SAHI outputs
df_annotations['bbox'] = df_annotations['bbox'].apply(ast.literal_eval) #fix the way bounding box is read
df_resolution = pd.read_csv("E:\\poweranalysis\\data\\circles\\BdA_2023_ind_imgs.csv") # output from step 2
def lowercase_extension(filename):
    name, ext = filename.rsplit('.', 1)
    return f'{name}.{ext.lower()}'

# Lowercase the extension part of filenames in one of the dataframes
df_resolution['filename'] = df_resolution['filename'].apply(lowercase_extension)

# Merge the two dataframes based on the 'filename' column
merged_df = df_annotations.merge(df_resolution, on='filename', how='inner')

# Image center in pixels
center_x_px, center_y_px = 3996, 2652  #IMPORTANT: Specify according to your sensor (1/2 width/height). This is for Sony RX1R II (Wingtra)

# Calculate the center point in meters
merged_df['center_x_m'] = center_x_px * merged_df['gsd']
merged_df['center_y_m'] = center_y_px * merged_df['gsd']

# Function to calculate distance from center
def calculate_distance_from_center(row):
    # Get the coordinates of the bounding box (x, y, width, height)
    x, y, width, height = row['bbox']

    # Calculate the center point of the bounding box in pixels
    bbox_center_x_px = x + (width / 2)
    bbox_center_y_px = y + (height / 2)

    # Calculate the center point of the bounding box in meters
    bbox_center_x_m = bbox_center_x_px * row['gsd']
    bbox_center_y_m = bbox_center_y_px * row['gsd']

    # Calculate the distance from the center of the image in meters
    distance_m = ((row['center_x_m'] - bbox_center_x_m)**2 + (row['center_y_m'] - bbox_center_y_m)**2)**0.5

    return distance_m

# Apply the function to the merged dataframe
merged_df['distance_from_center'] = merged_df.apply(calculate_distance_from_center, axis=1)

# Export the modified dataframe to CSV
savepath = "E:\\poweranalysis\\data\\circles\\"  # Desired output directory
merged_df.to_csv(savepath + "BdA2023_PowerAnalysisData2.csv", index=False)  # Desired filename


In [ ]:
#5a- Quick fix: GSD is in cm instead of m and you forgot

df_resolution = pd.read_csv("E:\\poweranalysis\\data\\circles\\StateLands2022_ind_imgs.csv") # output from step 2

df_resolution['gsd'] = df_resolution['gsd']/100

df_resolution.head()

,filename,altitude,gsd,transect_id,flight_id,site
0,20220110_Bernardo_0898.jpg,40,0.009040,1,1,Bernardo
1,20220110_Bernardo_0900.jpg,40,0.009040,1,1,Bernardo
2,20220110_Bernardo_0902.jpg,40,0.009040,1,1,Bernardo
3,20220110_Bernardo_0904.jpg,39,0.008814,1,1,Bernardo
4,20220110_Bernardo_0906.jpg,39,0.008814,1,1,Bernardo


In [100]:
#5b- Quick fix, if detections and independent image info isn't matching
df_resolution = pd.read_csv("E:\\poweranalysis\\data\\circles\\BdA_2023_ind_imgs.csv") # output from step 2
def lowercase_extension(filename):
    name, ext = filename.rsplit('.', 1)
    return f'{name}.{ext.lower()}'

# Lowercase the extension part of filenames in one of the dataframes
df_resolution['filename'] = df_resolution['filename'].apply(lowercase_extension)
df_resolution.head()

,filename,lat,long,agl,transect_id,gsd,land,unit
0,BdA_6s_Corridors_Flight_01_00002.jpg,33.852027,-106.855598,1482.361328,1,0.012945,yes,6
1,BdA_6s_Corridors_Flight_01_00004.jpg,33.852663,-106.855706,1481.988403,1,0.012897,yes,6
2,BdA_6s_Corridors_Flight_01_00006.jpg,33.853300,-106.855819,1482.775757,1,0.012999,yes,6
3,BdA_6s_Corridors_Flight_01_00008.jpg,33.853933,-106.855929,1484.390869,1,0.013207,yes,6
4,BdA_6s_Corridors_Flight_01_00010.jpg,33.854567,-106.856035,1482.853027,1,0.013009,yes,6


BELOW: Attempts on the process to use projected images for this process

In [14]:
#Derive independent (ie, non-overlapping) projected image footprints (whole image-- not the detection radius)
#This script works

import os
import shutil
from osgeo import gdal, osr

def calculate_overlap(tile1_path, tile2_path):
    # Open the raster datasets
    tile1_dataset = gdal.Open(tile1_path)
    tile2_dataset = gdal.Open(tile2_path)

    # Get the geotransform information (bounding box) of the tiles
    geotransform1 = tile1_dataset.GetGeoTransform()
    geotransform2 = tile2_dataset.GetGeoTransform()

    # Calculate overlapping area
    x_min1, y_max1 = geotransform1[0], geotransform1[3]
    x_max1, y_min1 = x_min1 + geotransform1[1] * tile1_dataset.RasterXSize, y_max1 + geotransform1[5] * tile1_dataset.RasterYSize

    x_min2, y_max2 = geotransform2[0], geotransform2[3]
    x_max2, y_min2 = x_min2 + geotransform2[1] * tile2_dataset.RasterXSize, y_max2 + geotransform2[5] * tile2_dataset.RasterYSize

    x_overlap = max(0, min(x_max1, x_max2) - max(x_min1, x_min2))
    y_overlap = max(0, min(y_max1, y_max2) - max(y_min1, y_min2))

    return x_overlap * y_overlap

def is_overlapping(tile_path, independent_tiles):
    for independent_tile_path in independent_tiles:
        if calculate_overlap(tile_path, independent_tile_path) > 0:
            return True
    return False

def find_non_overlapping_tiles(tile_directory):
    all_tile_paths = [os.path.join(tile_directory, file) for file in os.listdir(tile_directory) if file.endswith(".tif")]
    independent_tile_paths = [all_tile_paths[0]]

    for tile_path in all_tile_paths[1:]:
        if not is_overlapping(tile_path, independent_tile_paths):
            independent_tile_paths.append(tile_path)

    return independent_tile_paths

tile_directory = "E:\\poweranalysis\\data\\18a03"
independent_tile_paths = find_non_overlapping_tiles(tile_directory)

# Create a new directory for independent tiles
output_directory = "E:\\poweranalysis\\18a03_tiles"
os.makedirs(output_directory, exist_ok=True)

# Copy independent tiles to the output directory
for tile_path in independent_tile_paths:
    tile_filename = os.path.basename(tile_path)
    output_path = os.path.join(output_directory, tile_filename)
    shutil.copy(tile_path, output_path)

print("Independent tile paths have been copied to:", output_directory)


Independent tile paths have been copied to: E:\poweranalysis\18a03_tiles


In [45]:
#Translate image coordinates of SAHI detections to geographic ones 
#This still produces incorrect values :/

import os
import csv
import cv2
import numpy as np
import json  # Import the json module
from osgeo import gdal, osr

# Directory containing the GeoTIFF rasters
raster_dir = "E:\\poweranalysis\\data\\12c01_tiles\\"

# Define a function to convert image coordinates to geographic coordinates
def image_to_geographic_coordinates(image_x, image_y, geo_transform):
    x = geo_transform[0] + image_x * geo_transform[1]
    y = geo_transform[3] + image_y * geo_transform[5]
    return x, y

# Read the CSV file with bounding boxes
csv_file = "E:\\poweranalysis\\data\\12c01_coco_detections.csv"
output_csv_file = "E:\\poweranalysis\\data\\12c01_coco_detections_dist.csv"

with open(csv_file, "r") as file, open(output_csv_file, "w", newline='') as output_file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames + ["geo_x", "geo_y", "dist_m"]
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        image_id = row["image_id"]
        raster_filename = os.path.join(raster_dir, f"{image_id}")  # Construct raster filename from directory and image ID

        if not os.path.exists(raster_filename):
            print(f"Warning: Raster file '{raster_filename}' does not exist for Image ID '{image_id}'. Skipping.")
            continue

        # Load the GeoTIFF raster
        ds = gdal.Open(raster_filename)

        # Extract bounding box coordinates from the "bbox" column
        bbox_str = row["bbox"]
        bbox = json.loads(bbox_str)
        xmin, ymin, width, height = map(float, bbox)

        # Calculate the center of the bounding box in image coordinates
        center_x = xmin + (width / 2)
        center_y = ymin + (height / 2)

        # Calculate the geographic coordinates of the pixel corresponding to the bounding box center
        geo_transform = ds.GetGeoTransform()
        geo_x, geo_y = image_to_geographic_coordinates(center_x, center_y, geo_transform)

        # Calculate the distance between the geographic center and raster center
        raster_center_x = geo_transform[0] + (ds.RasterXSize / 2) * geo_transform[1]
        raster_center_y = geo_transform[3] + (ds.RasterYSize / 2) * geo_transform[5]
        distance = np.sqrt((geo_x - raster_center_x) ** 2 + (geo_y - raster_center_y) ** 2)

        # Add the new columns to the row
        row["geo_x"] = geo_x
        row["geo_y"] = geo_y
        row["dist_m"] = distance

        # Write the updated row to the new CSV
        writer.writerow(row)

        # Close the GeoTIFF dataset
        ds = None

# Optionally, you can rename the new CSV file to the original filename if desired
if os.path.exists(csv_file):
    os.remove(csv_file)  # Remove the original file if it exists
os.rename(output_csv_file, csv_file)  # Rename the new CSV file


In [50]:
#Translate outputs of SAHI detections to conform to the former YOLOv3 format used in Sa'doun's projection script
#This works

import pandas as pd
import os
from PIL import Image

# Input CSV file path
input_csv_file = "E:\\poweranalysis\\data\\12c01_unproj_coco_detections.csv"

# Output CSV file path
output_csv_file = "E:\\poweranalysis\\data\\output.csv"

# Directory where the images are located
image_directory = "E:\\poweranalysis\\data\\circles\\12c01_subset\\"

# Read the input CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Define a function to parse the "bbox" column
def parse_bbox(bbox_str):
    bbox = eval(bbox_str)
    xmin, ymin, width, height = bbox
    xmax = xmin + width
    ymax = ymin + height
    return xmin, ymin, xmax, ymax

# Apply the parsing function to the "bbox" column
df["xmin"], df["ymin"], df["xmax"], df["ymax"] = zip(*df["bbox"].apply(parse_bbox))

# Derive the "image_path" by adding "image_id" to the image directory
df["image_path"] = image_directory + df["image_id"]

# Use "category_id" for "label" and "score" for "confidence"
df.rename(columns={"category_id": "label", "score": "confidence"}, inplace=True)

# Extract "x_size" and "y_size" by opening each image
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

df["x_size"], df["y_size"] = zip(*df["image_path"].apply(get_image_size))

# Create the output DataFrame with the desired columns
output_df = df[["image_id", "image_path", "xmin", "ymin", "xmax", "ymax", "label", "confidence", "x_size", "y_size"]]
output_df.columns = ["image", "image_path", "xmin", "ymin", "xmax", "ymax", "label", "confidence", "x_size", "y_size"]

# Save the output DataFrame to a new CSV file
output_df.to_csv(output_csv_file, index=False)

print(f"Conversion complete. Output saved to {output_csv_file}")

Conversion complete. Output saved to E:\poweranalysis\data\output.csv
